## PDF Yüklemesinin Gerçekleştirilmesi

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("calikusu.pdf")
data = loader.load()

len(data)

298

## Veriyi parçalara ayırma(Chunking İşlemi)

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

## Chunk_Size 1000 demek her parçanın maksimum 1000 karakter uzunluğunda olacağını belirtir.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total",len(docs))

Total 935


In [6]:
docs[7]

Document(metadata={'producer': 'calibre 1.22.0 [http://calibre-ebook.com]', 'creator': 'calibre 1.22.0 [http://calibre-ebook.com]', 'creationdate': '2018-06-07T15:24:58+00:00', 'author': 'Reşat Nuri Güntekin', 'keywords': 'Roman, Türk Klasik, Cumhuriyet Dönemi', 'title': 'Çalıkuşu', 'source': 'calikusu.pdf', 'total_pages': 298, 'page': 7, 'page_label': '8'}, page_content='BİRİNCİ KISIM\nDÖRDÜNCÜ sınıftaydım. Yaşı\nm oniki kadar olmalı. Fransızca muallimimiz Sör Aleksi, bir gün\nbize yazı vazifesi vermişti. “Hayattaki ilk hatıralarınızı yazmaya çalışın. Bakalım neler bulacaksınız?\nSizin için güzel bir hayat temini olur,” demişti.\nHiç unutmam; yaramazlığımdan, gevezeliğimden bıkan öğretmenler, o sınıfta beni arkadaşlarımdan\nayırmışlar, bir köşede tek kişilik bir küçük sıraya oturtmuşlardı.\nMüdirenin söylediğine göre, ders esnasında komşularımı lakırdıya tutmamayı, uslu uslu muallimi\ndinlemeyi öğreninceye kadar orada bir sürgün hayatı geçirmeye mahkûmdum.\nBir yanımda kocaman bir tah

## OpenAI Generative AI Embeddings'i kullanarak Embedding Oluşturma İşlemi

In [7]:
from langchain_chroma import Chroma
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings



In [8]:
load_dotenv()

True

In [9]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

C:\Users\ruveyda.cetin\AppData\Local\Temp\ipykernel_33952\2421187937.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


## ChromaDB Üzerine Kayıt İşlemi

In [10]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

In [15]:
retriever = vectorstore.as_retriever(
    search_type="similarity", search_kwargs={"k": 50}
)

In [16]:
retieved_docs = retriever.invoke("Feride kimdir?")

In [17]:
len(retieved_docs)

50

In [18]:
print(retieved_docs[5].page_content)

-Ben, Feride Hanımefendi ile görüşmek istiyorum. Hafifçe irkildim, gülmemek için başımı eğdim,
ilk defa işittiğim bu “hanımefendi” sözü bana öyle tuhaf geliyordu ki... Feride Hanımefendi’nin ben
olduğumu mümkün değil, söylemeye cesaret edemeyecektim. Dudaklarımı ısırarak:
-Çok âlâ hanımefendi, dedim. Buyurun, lütfen içeri; köşkten sorarsanız size Feride Hanım’ı
çağırırlar.
Siyah çarşaflı kadın, kapıdan girmiş yanıma yaklaşmıştı:
-Size tesadüfüm çok iyi oldu, çocuğum, dedi. Sizden bir yardım rica edeceğim. Benim Feride
Hanım’la yalnız olarak konuşmama delalet edeceksiniz. Mümkünse kimsenin bundan haberi
olmamalı.
Hayretle yüzüne baktım. Ortalık kararmış olduğu ve peçesini hâlâ açmadığı için yüzünü fark
edemiyordum. Hafif bir tereddütten sonra:
-Hanımefendi, dedim. Acayip bir kıyafette olduğum için birdenbire cesaret edemedim. Fakat
Feride benim. Kadın, hafif heyecanlandı:
-Kâmran Bey’le evlenecek Feride Hanım mı?
-Köşkte bir tane Feride var hanımefendi, diye gülümsedim.


## OpenAI API Yapısını Kullanarak LLM Tetikleme İşlemleri

- Düşük Değerler (0.1-0.4) : Daha kesin ve daha tutarlı cevaplar verilir. Model daha tamin edilebilir hale gelir. 
- Orta Değerler (0.5-0.7) : Hem mantıklı hem de yaratıcı cevaplar verilir. 
- Yüksek Değerler (0.7-1.0): Daha rastgele ve yaratıcı, ancak bazen tutarsız yanıtlar verebilir

In [19]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.3,
    max_tokens=500
)

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


In [31]:
system_prompt = (
    "📖 Sen, Reşat Nuri Güntekin’in *Çalıkuşu* romanı üzerine uzman, cana yakın bir edebi asistansın. "
    "Görevin, kısa ama etkileyici analizler yapmak; karakterleri, temaları ve duyguları samimi bir dille yorumlamak 😊 "
    "Sadece bu roman hakkında konuş 🛑 Başka kitaplardan bahsetme. "
    "Yanıtların Türkçe olsun 🇹🇷 ve açıklamalarını emojilerle zenginleştir 📌"
    "\n\n"
    "{context}"
)


In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

## Soru-Cevap Zinciri Oluşturma(LLM+PROMPT)

In [33]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)

## RAG Zinciri Oluşturma(RAG+LLM Entegrasyonun Gerçekleşmesi)

In [34]:
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

## Kullanıcı Sorgusunu Çalıştırarak Cevap Üretme İşlemi

In [43]:
response = rag_chain.invoke({"input":"Peki bu kitabı özetlesen nasıl özetlersin?"})

In [44]:
print(response)

{'input': 'Peki bu kitabı özetlesen nasıl özetlersin?', 'context': [Document(id='04cef9d9-8323-4927-b5c3-f8a6a3b35e55', metadata={'author': 'Reşat Nuri Güntekin', 'creationdate': '2018-06-07T15:24:58+00:00', 'creator': 'calibre 1.22.0 [http://calibre-ebook.com]', 'keywords': 'Roman, Türk Klasik, Cumhuriyet Dönemi', 'page': 236, 'page_label': '237', 'producer': 'calibre 1.22.0 [http://calibre-ebook.com]', 'source': 'calikusu.pdf', 'title': 'Çalıkuşu', 'total_pages': 298}, page_content='Hayretle yüzüne baktım. O, memnuniyetle gülerek:\n-Hoşuna gidecek şeyi nasıl keşfettim. Bizim “Alacakaya”daki çiftliği ben, otuz kırk kişilik bir\nyetimhane şekline sokuyorum. Etrafta bulduğumuz kimsesiz çocukları oraya toplayacağız. Ben\ndoktorluk edeceğim, sen hocalık ve analık.\nBu satırları, nekahat günlerimi geçirdiğim odanın penceresi önünde yazıyorum. Bahçedeki dallarda\nhiç durmayan bir kuru yaprak yağmuru yağıyor.\nAğaçların çıplak kollarından döktüğü bu yapraklardan bazılarını rüzgâr, pencereden

In [45]:
print(response["answer"])

*Çalıkuşu*, Reşat Nuri Güntekin'in kaleme aldığı, genç bir öğretmen olan Feride'nin hayatını ve içsel yolculuğunu anlatan etkileyici bir romandır. Feride, güzelliği ve neşesiyle çevresindekileri etkileyen, ancak aşk hayatında hayal kırıklıkları yaşayan bir genç kadındır. Nişanlısı Kâmran'ın ihanetine uğradığını düşündüğünde, İstanbul'dan kaçar ve Anadolu'nun çeşitli köylerinde öğretmenlik yapmaya başlar. 

Bu süreçte Feride, hem Anadolu'nun zorluklarıyla hem de kendi içsel çatışmalarıyla yüzleşir. Kendi ayakları üzerinde durmayı öğrenirken, karşılaştığı zorluklar ve insanlarla olan ilişkileri sayesinde olgunlaşır. Roman, Feride'nin içsel gücünü, bağımsızlığını ve idealizmini keşfetmesini anlatırken, aynı zamanda dönemin toplumsal yapısını ve Anadolu'nun gerçeklerini de gözler önüne serer. 

Feride'nin hikayesi, aşk, ihanet, yalnızlık ve kendini bulma temaları etrafında şekillenir. Sonunda, geçmişiyle yüzleşip, gerçekleri öğrendiğinde, hayatına yeni bir yön vermeye karar verir. *Çalıkuş